In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import numpy as np

In [7]:
baseball = pd.read_csv('best_model/engineered_data.csv')

In [8]:
X = baseball.drop(['Salary'], axis = 1)
y = baseball['Salary']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .25, random_state = 621)

In [9]:
cat_columns = ['Tm', 'Lg', 'Acquired', 'Bat']
num_columns = [col for col in X.columns if col not in cat_columns + ['Pos_C', 'Pos_1B', 'Pos_2B', 'Pos_3B', 'Pos_SS', 'Pos_OF']]

cat_transformer = Pipeline(
    steps = [
        ('onehot', OneHotEncoder(handle_unknown = 'ignore'))
    ]
)

num_transformer = Pipeline(
    steps = [
        ('scale', StandardScaler())
    ]
)

preprocessor = ColumnTransformer(
    transformers = [
        ('cont', num_transformer, num_columns),
        ('cat', cat_transformer, cat_columns)
    ], remainder = 'passthrough'
)

X_transform = preprocessor.fit_transform(X)

selected_features = np.concatenate([
    np.array(num_columns),
    np.array(preprocessor.transformers_[1][1]['onehot'].get_feature_names_out(cat_columns)),
    np.array(['Pos_C', 'Pos_1B', 'Pos_2B', 'Pos_3B', 'Pos_SS', 'Pos_OF'])
])


In [10]:
from keras.models import Sequential
from keras.layers import Dropout, Dense
from bayes_opt import BayesianOptimization
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping
import warnings
import joblib

warnings.filterwarnings('ignore')


2024-04-17 18:29:57.978012: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_transform, y, test_size = .2, random_state = 621)

In [12]:
def dnn_model_score(neurons, dropout_rate, learning_rate, epochs, batch_size, patience, num_layers, **layer_neurons):
    model = Sequential()
    model.add(Dense(int(neurons), activation='relu', input_shape = (X_train.shape[1],)))
    model.add(Dropout(dropout_rate))

    for i in range(1, int(num_layers) + 1):
        model.add(Dense(int(layer_neurons[f'layer_neurons_{i}']), activation='relu'))
        model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation = 'linear'))

    optimizer = Adam(learning_rate = learning_rate)
    model.compile(loss = 'mean_squared_error', optimizer = optimizer)

    es = EarlyStopping(monitor = 'val_loss', patience = int(patience), restore_best_weights = True)

    model.fit(X_train, y_train, validation_split = .2, epochs = int(epochs), batch_size = int(batch_size), callbacks = es, verbose = 0)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    return -mse

pbounds = {'neurons': (32, 256),
           'dropout_rate': (0.0, 0.5),
           'learning_rate': (0.01, 1),
           'epochs' : (100, 500),
           'batch_size' : (32, 500),
           'patience' : (20, 50),
           'num_layers': (1, 5)}

for i in range(1, 6):
    pbounds[f'layer_neurons_{i}'] = (32, 256)

optimizer = BayesianOptimization(f = dnn_model_score, pbounds = pbounds, random_state = 42)

optimizer.maximize(init_points = 5, n_iter = 10)

best_params = optimizer.max['params']
print("Best Hyperparameters:", best_params)


|   iter    |  target   | batch_... | dropou... |  epochs   | layer_... | layer_... | layer_... | layer_... | layer_... | learni... |  neurons  | num_la... | patience  |
-------------------------------------------------------------------------------------------------------------------------------------------------------------------------
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
| 1         | -2.119e+1 | 207.3     | 0.4754    | 392.8     | 166.1     | 66.95     | 66.94     | 45.01     | 226.0     | 0.6051    | 190.6     | 1.082     | 49.1      |
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
| 2         | -2.333e+1 | 421.6     | 0.1062    | 172.7     | 73.08     | 100.2     | 149.5     | 128.8     | 97.24     | 0.6157    | 63.25     | 2.169     | 30.99     |
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step
| 3         | -2.185e+1 | 245.4     | 0.3926    | 179.9     | 147.2     | 164.7     | 42.4      | 168.1     | 70.2      | 0.0744    | 244.6     | 4.863     | 44.25     |
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms

In [13]:
batch_size = int(best_params['batch_size'])
dropout_rate = best_params['dropout_rate']
epochs = int(best_params['epochs'])
neurons = []
neurons.append(int(best_params['neurons']))
neurons.append(int(best_params['layer_neurons_1']))
neurons.append(int(best_params['layer_neurons_2']))
neurons.append(int(best_params['layer_neurons_3']))
neurons.append(int(best_params['layer_neurons_4']))
neurons.append(int(best_params['layer_neurons_5']))
learning_rate = best_params['learning_rate']
num_layers = int(best_params['num_layers'])
patience = int(best_params['patience'])

In [14]:
model = Sequential()

model.add(Dense(name = 'Dense1', units = neurons[0], input_dim = X_train.shape[1], activation = 'relu'))
model.add(Dropout(name = 'Dropout1', rate = dropout_rate))

for i in range(1, num_layers + 1):
    model.add(Dense(name = f'Dense{i + 1}', units = neurons[i], activation = 'relu'))
    model.add(Dropout(name = f'Dropout{i + 1}', rate = dropout_rate))

model.add(Dense(name = 'Output', units = 1, activation = 'linear'))

optimizer = Adam(learning_rate = learning_rate)

model.compile(optimizer = optimizer, loss = 'mean_squared_error')

model.summary()

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ Dense1 (Dense)                  │ (None, 190)            │        14,820 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout1 (Dropout)              │ (None, 190)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dense2 (Dense)                  │ (None, 166)            │        31,706 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Dropout2 (Dropout)              │ (None, 166)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Output (Dense)                  │ (None, 1)              │           167 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 46,693 (182.39 KB)

 Trainable params: 46,693 (182.39 KB)

 Non-trainable params: 0 (0.00 B)

In [15]:
es = EarlyStopping(monitor = 'val_loss', patience = int(patience), restore_best_weights = True)

history = model.fit(X_train, y_train, validation_split = .2, batch_size = batch_size, epochs = epochs, callbacks = es, verbose = 2)

Epoch 1/392
16/16 - 4s - 245ms/step - loss: 48168855142400.0000 - val_loss: 32053166866432.0000
Epoch 2/392
16/16 - 0s - 12ms/step - loss: 32669312221184.0000 - val_loss: 28785554489344.0000
Epoch 3/392
16/16 - 0s - 16ms/step - loss: 30067356860416.0000 - val_loss: 26202389610496.0000
Epoch 4/392
16/16 - 0s - 19ms/step - loss: 28963168583680.0000 - val_loss: 25250651701248.0000
Epoch 5/392
16/16 - 0s - 24ms/step - loss: 27783900168192.0000 - val_loss: 24963673227264.0000
Epoch 6/392
16/16 - 0s - 13ms/step - loss: 27591247396864.0000 - val_loss: 25080568479744.0000
Epoch 7/392
16/16 - 0s - 21ms/step - loss: 27414763667456.0000 - val_loss: 24696877744128.0000
Epoch 8/392
16/16 - 0s - 17ms/step - loss: 26610128388096.0000 - val_loss: 24780031918080.0000
Epoch 9/392
16/16 - 0s - 24ms/step - loss: 26745212239872.0000 - val_loss: 24828731981824.0000
Epoch 10/392
16/16 - 0s - 19ms/step - loss: 26856914944000.0000 - val_loss: 23967656050688.0000
Epoch 11/392
16/16 - 1s - 32ms/step - loss: 2530

In [16]:
test_preds = model.predict(X_test)
train_preds = model.predict(X_train)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step
127/127 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step


In [17]:
train_rmse = mean_squared_error(y_train, train_preds)
test_rmse = mean_squared_error(y_test, test_preds)

print('Deep Neural Network Metrics:')
print(f'Train RMSE: {np.sqrt(train_rmse)}')
print(f'Test RMSE: {np.sqrt(test_rmse)}')

Deep Neural Network Metrics:
Train RMSE: 3085814.808687137
Test RMSE: 4665029.528188211


In [20]:
# save the model, pca, and preprocessor so that new data can be fit using the same criteria
#model.save('best_model/best_model.keras')
#joblib.dump(pca, 'dnn/pca45.joblib')
#joblib.dump(preprocessor, 'best_model/best_preprocssor.joblib')

['best_model/y_test.joblib']